In [1]:
import os
import pandas as pd

raw_data_dir = "input_raw_data"
transactions_df = pd.read_csv(os.path.join(raw_data_dir, "transaction.csv"))
identity_df = pd.read_csv(os.path.join(raw_data_dir, "identity.csv"))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
transactions_df.head(5)

,TransactionID,TransactionDT,card_no,card_type,email_domain,ProductCD,TransactionAmt,isFraud
0,9c90c7e2-2600-4628-a868-179287eee169,2012-01-15 00:00:20,30545481171260,JCB 16 digit,smith-henry.info,T,1198,0
1,02951e68-f8ff-4f00-b515-5df8bcb0edcb,2012-01-15 00:00:44,6549624810102543,VISA 13 digit,yahoo.com,Y,409,0
2,17071ccd-3c3d-476a-a3cd-b73638a803d4,2012-01-15 00:02:18,4792410992636220,JCB 15 digit,hotmail.com,Y,1101,0
3,5288f832-6673-473a-80f6-fdb98dd99278,2012-01-15 00:03:01,3573503852773765,VISA 16 digit,hotmail.com,Y,2228,0
4,e37a6dd1-182e-4b87-8b8e-7dd9bf78cea0,2012-01-15 00:03:24,3597910599495184,Diners Club / Carte Blanche,gmail.com,L,2393,0


In [3]:
identity_df.head(5)

,TransactionID,IpAddress,PhoneNo,DeviceID
0,f9980c5d-e8bf-4431-9ddf-15b4e7a9d7ff,104.32.122.67,7993004217,657817920030
1,011955b3-14fa-4342-85f4-3a9c3759b7d7,164.180.250.14,6712423669,8984812524268
2,a61677ac-a0cb-4113-9409-87bcfe7e6e44,99.31.120.70,+1-353-046-6749x00004,3092060466768
3,f46e41fc-5d06-4f60-b730-a5d7ecec90f1,10.225.43.49,125-534-6633x1775,2516617099683
4,4cc7b6f5-9579-4149-80d8-8cb6c807ee6c,218.46.8.47,(374)448-9917,7683358940269


In [4]:
full_identity_df = transactions_df.merge(identity_df, on="TransactionID", how="left")

# drop transcations time column as it is not useful for constructing graph.
full_identity_df.drop(["TransactionDT"], axis=1, inplace=True)

# Re-arange the order of column names for better visualization
full_identity_df = full_identity_df[
    [
        "TransactionID",
        "card_no",
        "card_type",
        "email_domain",
        "IpAddress",
        "PhoneNo",
        "DeviceID",
        "ProductCD",
        "TransactionAmt",
        "isFraud",
    ]
]
full_identity_df.head(5)

,TransactionID,card_no,card_type,email_domain,IpAddress,PhoneNo,DeviceID,ProductCD,TransactionAmt,isFraud
0,9c90c7e2-2600-4628-a868-179287eee169,30545481171260,JCB 16 digit,smith-henry.info,121.92.230.58,+1-038-395-3162x55186,7163249193818,T,1198,0
1,02951e68-f8ff-4f00-b515-5df8bcb0edcb,6549624810102543,VISA 13 digit,yahoo.com,118.244.38.209,001-923-541-7445x63231,3807588936118,Y,409,0
2,17071ccd-3c3d-476a-a3cd-b73638a803d4,4792410992636220,JCB 15 digit,hotmail.com,66.25.190.213,955-326-1614,9673128747953,Y,1101,0
3,5288f832-6673-473a-80f6-fdb98dd99278,3573503852773765,VISA 16 digit,hotmail.com,42.93.10.194,+1-339-892-9034x139,9965894358908,Y,2228,0
4,e37a6dd1-182e-4b87-8b8e-7dd9bf78cea0,3597910599495184,Diners Club / Carte Blanche,gmail.com,154.168.168.7,278.713.5654,7523612045351,L,2393,0
